ANN Final Project
Authors: Caleb Johnson, Gabe Schwartz, Evan Kates
Network Module

In [55]:
# import data from csv file
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 2: Load the data
df = pd.read_csv('training.csv')

# Step 3: Define features (X) and targets (y)
# Optimized rookie year features
feature_cols = ['GP_r', 'MIN_r', 'FG_PCT_r', 'REB_r', 'AST_r', 'PTS_r', 'TOV_r']

X = df[feature_cols]

# Sophomore stats to predict
target_cols = ['PTS_s', 'REB_s', 'AST_s']
y = df[target_cols]

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Feature scaling (important for neural nets)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 6: Build the model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],),
                       kernel_regularizer=regularizers.l1(0.001)),  # L1 regularization
    keras.layers.Dense(64, activation='relu',
                       kernel_regularizer=regularizers.l1(0.001)),
    keras.layers.Dense(3)  # Predicting 3 values: PTS, REB, AST
])

# Define early stopping
early_stop = EarlyStopping(
    monitor='val_loss',   # Watch the validation loss
    patience=10,          # Number of epochs to wait after no improvement
    restore_best_weights=True # After stopping, roll back to the best model
)

model.compile(optimizer='adam', loss=losses.Huber(), metrics=['mae'])


/Users/calebjohnson/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [56]:
# Step 7: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=300,
    batch_size=32,
    callbacks=[early_stop]
)

# Step 8: Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae}")

# Step 9: Make predictions
predictions = model.predict(X_test)

# Show some example predictions
for i in range(5):
    print(f"Predicted: {predictions[i]} | Actual: {y_test.iloc[i].values}")

Epoch 1/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 5.1517 - mae: 4.8283 - val_loss: 4.8067 - val_mae: 4.4815
Epoch 2/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.2942 - mae: 3.9629 - val_loss: 4.0888 - val_mae: 3.7669
Epoch 3/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.5914 - mae: 3.2600 - val_loss: 3.4058 - val_mae: 3.0821
Epoch 4/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.8703 - mae: 2.5445 - val_loss: 2.7994 - val_mae: 2.4828
Epoch 5/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.4329 - mae: 2.1207 - val_loss: 2.3300 - val_mae: 2.0289
Epoch 6/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.1590 - mae: 1.8599 - val_loss: 2.0577 - val_mae: 1.7692
Epoch 7/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.9597 - mae: 1.6800 - val_loss: 1.9455 - val_mae: 1.6882
Epoch 8/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.9464 - mae: 1.6902 - val_loss: 1.8782 - val_mae: 1.6490
Epoch 9/300
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.82

In [57]:
from scipy import stats

# 1. Predict on test data
y_pred = model.predict(X_test)

# 2. Calculate residuals
errors = y_test.values - y_pred

# 3. Estimate standard deviation of residuals
std_dev = np.std(errors, axis=0)

# 4. Compute 95% confidence intervals
# 1.96 standard deviations covers ~95% for a normal distribution

conf_interval = 1.645 * std_dev

print("We are 90% confident that the stats will be within these ranges:")
print(f"Points: ±{conf_interval[0]:.2f}")
print(f"Rebounds: ±{conf_interval[1]:.2f}")
print(f"Assists: ±{conf_interval[2]:.2f}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
We are 90% confident that the stats will be within these ranges:
Points: ±4.60
Rebounds: ±2.68
Assists: ±1.27
